In [1]:
!pip install pysam
import pysam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 31.0 MB/s eta 0:00:00


In [2]:
!apt-get install bcftools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libhts3 libhtscodecs2
Suggested packages:
  python3-numpy python3-matplotlib texlive-latex-recommended
The following NEW packages will be installed:
  bcftools libhts3 libhtscodecs2
0 upgraded, 3 newly installed, 0 to remove and 33 not upgraded.
Need to get 1,140 kB of archives.
After this operation, 3,448 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhtscodecs2 amd64 1.1.1-3 [53.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhts3 amd64 1.13+ds-2build1 [390 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 bcftools amd64 1.13-1 [697 kB]
Fetched 1,140 kB in 1s (1,211 kB/s)
Selecting previously unselected package libhtscodecs2:amd64.
(Reading database ... 121749 files and directories currently installed.)
Preparing to unpack .../libhtscodecs2_1.

In [3]:
!wget 'https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr19.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz'

--2024-02-19 15:27:10--  https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr19.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.167
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344793450 (329M) [application/x-gzip]
Saving to: ‘ALL.chr19.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz’

ALL.chr19.phase3_sh 100%[===================>] 328.82M   709KB/s    in 8m 10s  

2024-02-19 15:35:21 (687 KB/s) - ‘ALL.chr19.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz’ saved [344793450/344793450]



In [4]:
!bcftools index 'ALL.chr19.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz'


In [11]:
!bcftools view -r 19:3610645-3626815 ALL.chr19.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz > cactin.vcf.gz

In [12]:
!bcftools view --min-af 0.01 -o maf.vcf.gz cactin.vcf.gz

In [13]:
!bcftools view --max-alleles 2 -o bialles.vcf.gz maf.vcf.gz

In [14]:
!bcftools view -v snps bialles.vcf.gz > snp_cactin.vcf.gz

In [15]:
!bcftools query -l bialles.vcf.gz | wc -l ; bcftools view -v snps bialles.vcf.gz | grep -v "^#" | wc -l

2504
95


In [18]:
def Get_Genotypes(vcf_file_path):
    genotypes = []
    reader=pysam.VariantFile(vcf_file_path, 'r')
    for i in range(len(reader.header.samples)):
        genotypes.append([])
    for record in reader:
        for i, sample in enumerate(record.samples.values()):
            genotype = sample['GT']
            genotypes[i].append(genotype)
    return genotypes

haplotype_list= []
gen_hap_list = []
genotype_list = []
vcf_file_path = '/content/bialles.vcf.gz'
genotypes = Get_Genotypes(vcf_file_path)
for i, genotypes in enumerate(genotypes):
    h1 = ''.join([str(t[0]) for t in genotypes])
    h2 = ''.join([str(t[1]) for t in genotypes])
    gen = ''.join([str(t[0]+t[1]) for t in genotypes])
    gen_hap_list.append((h1, h2, gen))
    haplotype_list.append(h1)
    haplotype_list.append(h2)
    genotype_list.append(gen)


haplo_path = "haplotype.txt"
geno_path = "genotype.txt"
gen_hap_path = "gen_hap.txt"
def save(path, str_list):
  f=open(path,"w")
  for string in str_list:
    f.write(string + "\n")

save(haplo_path, haplotype_list)
save(geno_path, genotype_list)

gen_hap_combine_list=[]
for item in gen_hap_list:
  gen_hap_combine_list.append(item[0]+" "+item[1]+" "+item[2])
save(gen_hap_path, gen_hap_combine_list)